# Import des outils / jeu de données

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import prince
import seaborn as sns
from sklearn.cluster import (
    DBSCAN,
    OPTICS,
    AffinityPropagation,
    AgglomerativeClustering,
    KMeans,
    MeanShift,
)
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    calinski_harabasz_score,
    davies_bouldin_score,
    silhouette_score,
)
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import RobustScaler, StandardScaler

In [ ]:
SEED = 0

In [ ]:
np.random.seed(SEED)
sns.set_theme()

In [ ]:
df = pd.read_csv(
    "data/data-cleaned-feature-engineering.csv",
    sep=",",
    index_col="ID",
    parse_dates=True,
)

In [ ]:
composantes_acp = pd.read_csv("data/composantes_acp.csv", index_col="ID")
composantes_acm = pd.read_csv("data/composantes_acm.csv", index_col="ID")

## Variables globales

In [ ]:
var_numeriques = [
    "Year_Birth",
    "Income",
    "Recency",
    "MntWines",
    "MntFruits",
    "MntMeatProducts",
    "MntFishProducts",
    "MntSweetProducts",
    "MntGoldProds",
    "NumDealsPurchases",
    "NumWebPurchases",
    "NumCatalogPurchases",
    "NumStorePurchases",
    "NumWebVisitsMonth",
]

In [ ]:
var_categoriques = [
    "Education",
    "Marital_Status",
    "Kidhome",
    "Teenhome",
    "AcceptedCmp1",
    "AcceptedCmp2",
    "AcceptedCmp3",
    "AcceptedCmp4",
    "AcceptedCmp5",
    "Response",
]

var_categoriques_extra = ["NbAcceptedCampaigns", "HasAcceptedCampaigns", "NbChildren"]

var_categoriques_fe = var_categoriques + var_categoriques_extra

# Clustering

## Préparation des données

Nous commencer par fusionner les variables quantitatives et les coordonnées des individus dans l'ACM.

In [ ]:
X_clust = pd.concat((df[var_numeriques], composantes_acm), axis=1)

In [ ]:
X_clust.head()

In [ ]:
preprocessor = ColumnTransformer(
    remainder="passthrough",
    transformers=[
        ("scaler", RobustScaler(), var_numeriques),
    ],
)

In [ ]:
scaler = RobustScaler()
df_apres_scale = pd.DataFrame(
    preprocessor.fit_transform(X_clust),
    columns=X_clust.columns,
    index=df.index,
)

In [ ]:
df_apres_scale.head()

In [ ]:
df_avec_clusters = df_apres_scale.copy()

## Différents algorithmes de clusters

Nous choisissons de tester 2 types de modèles de clustering :
1) les modèles à choix du nombre de clusters (KMeans, GMM, CAH)
2) les modèles qui décident du nombre de clusters (OPTICS, MeanShift et AffinityPropagation)

Cela nous permettra de comparer le nombre de clusters donné par les seconds algorithmes.

Pour les modèles pour lesquels il faut choisir le nombre de clusters, nous décidons de tester des clusters de taille 2 à 5 (inclus), car un trop grand nombre de clusters serait plus difficile à interpréter pour l'équipe marketing dans un premier temps.

In [ ]:
NB_CLUSTER_MIN = 2
NB_CLUSTER_MAX = 6  # non inclus

In [ ]:
dict_kmeans = {
    f"KMeans{i}": KMeans(n_clusters=i, random_state=SEED)
    for i in range(NB_CLUSTER_MIN, NB_CLUSTER_MAX)
}

dict_gmm = {
    f"GMM{i}": GaussianMixture(
        n_components=i, covariance_type="full", random_state=SEED
    )
    for i in range(NB_CLUSTER_MIN, NB_CLUSTER_MAX)
}

dict_cah_ward = {
    f"CAH (Ward) {i}": AgglomerativeClustering(n_clusters=i)
    for i in range(NB_CLUSTER_MIN, NB_CLUSTER_MAX)
}

dict_cah_average = {
    f"CAH (average linkage) {i}": AgglomerativeClustering(
        n_clusters=i, linkage="average"
    )
    for i in range(NB_CLUSTER_MIN, NB_CLUSTER_MAX)
}

dict_cah_simple = {
    f"CAH (single linkage) {i}": AgglomerativeClustering(n_clusters=i, linkage="single")
    for i in range(NB_CLUSTER_MIN, NB_CLUSTER_MAX)
}

dict_cah_complete = {
    f"CAH (complete linkage) {i}": AgglomerativeClustering(
        n_clusters=i, linkage="complete"
    )
    for i in range(NB_CLUSTER_MIN, NB_CLUSTER_MAX)
}

In [ ]:
model_clusters = {
    **dict_kmeans,
    **dict_gmm,
    **dict_cah_ward,
    **dict_cah_average,
    **dict_cah_simple,
    **dict_cah_complete,
    "OPTICS": OPTICS(),
    "MeanShift": MeanShift(),
    "AffinityPropagation": AffinityPropagation(random_state=SEED),
}

In [ ]:
# todo: faire une liste des algos ???

In [ ]:
cluster_metrics = []

for (model_name, model) in model_clusters.items():
    if model.__class__ == GaussianMixture:  # cas particulier du mélange gaussien
        df_avec_clusters[model_name] = model.fit_predict(df_apres_scale)
    else:
        model.fit(df_apres_scale)
        df_avec_clusters[model_name] = model.labels_

    df_avec_clusters[model_name] = pd.Categorical(
        df_avec_clusters[model_name].astype(str)
    )

    nb_clusters = df_avec_clusters[model_name].nunique()

    repartition = list(
        df_avec_clusters[model_name].value_counts(normalize=True).round(2).astype(str)
    )  # todo: enlever astype(str) si ça sert à rien (tester)

    cluster_metrics.append(
        [
            model_name,
            nb_clusters,
            " | ".join(repartition),
            silhouette_score(
                df_apres_scale, df_avec_clusters[model_name], random_state=SEED
            ),  # proche de 1 = mieux
            calinski_harabasz_score(
                df_apres_scale,
                df_avec_clusters[model_name],
            ),  # plus élevé, mieux c'est
            davies_bouldin_score(
                df_apres_scale, df_avec_clusters[model_name]
            ),  # proche de 0 = mieux
        ]
    )

In [ ]:
pd.DataFrame(
    cluster_metrics,
    columns=[
        "Algorithme de clustering",
        "Nombre de clusters",
        "Répartition",
        "Silhouette",
        "Calinski-Harabasz",
        "Davies-Bouldin",
    ],
)

|    | Algorithme de clustering   | Nombre de clusters | Répartition                                         | Silhouette | Calinski-Harabasz | Davies-Bouldin |
|:---|:---------------------------|:-------------------|:----------------------------------------------------|:-----------|:------------------|:---------------|
| 0  | KMeans2                    | 2                  | 0.98 \| 0.02                                        | 0.659488   | 582.622386        | 0.615865       |
| 1  | KMeans3                    | 3                  | 0.67 \| 0.31 \| 0.02                                | 0.244051   | 507.123539        | 1.668756       |
| 2  | KMeans4                    | 4                  | 0.6 \| 0.27 \| 0.11 \| 0.02                         | 0.259113   | 531.915899        | 1.472760       |
| 3  | KMeans5                    | 5                  | 0.56 \| 0.18 \| 0.13 \| 0.11 \| 0.02                | 0.248015   | 455.952230        | 1.696608       |
| 4  | GMM2                       | 2                  | 0.52 \| 0.48                                        | 0.129703   | 214.211136        | 2.578815       |
| 5  | GMM3                       | 3                  | 0.51 \| 0.35 \| 0.14                                | 0.172544   | 265.725759        | 2.043114       |
| 6  | GMM4                       | 4                  | 0.49 \| 0.35 \| 0.14 \| 0.02                        | 0.200750   | 495.185144        | 1.598463       |
| 7  | GMM5                       | 5                  | 0.43 \| 0.27 \| 0.15 \| 0.12 \| 0.02                | 0.145860   | 419.354348        | 1.630245       |
| 8  | CAH \(Ward\) 2             | 2                  | 0.98 \| 0.02                                        | 0.646087   | 576.969331        | 0.645240       |
| 9  | CAH \(Ward\) 3             | 3                  | 0.88 \| 0.1 \| 0.02                                 | 0.303866   | 503.041927        | 1.085627       |
| 10 | CAH \(Ward\) 4             | 4                  | 0.69 \| 0.19 \| 0.1 \| 0.02                         | 0.238196   | 464.335686        | 1.501346       |
| 11 | CAH \(Ward\) 5             | 5                  | 0.59 \| 0.19 \| 0.1 \| 0.09 \| 0.02                 | 0.229092   | 411.926556        | 1.614228       |
| 12 | CAH \(average linkage\) 2  | 2                  | 0.99 \| 0.01                                        | 0.756695   | 391.532573        | 0.305244       |
| 13 | CAH \(average linkage\) 3  | 3                  | 0.98 \| 0.02 \| 0.01                                | 0.555290   | 347.021115        | 0.520527       |
| 14 | CAH \(average linkage\) 4  | 4                  | 0.96 \| 0.02 \| 0.02 \| 0.01                        | 0.505373   | 310.852921        | 0.671113       |
| 15 | CAH \(average linkage\) 5  | 5                  | 0.96 \| 0.02 \| 0.02 \| 0.01 \| 0.0                 | 0.505079   | 235.416573        | 0.575378       |
| 16 | CAH \(single linkage\) 2   | 2                  | 0.99 \| 0.01                                        | 0.756695   | 391.532573        | 0.305244       |
| 17 | CAH \(single linkage\) 3   | 3                  | 0.99 \| 0.01 \| 0.0                                 | 0.753557   | 198.871039        | 0.266272       |
| 18 | CAH \(single linkage\) 4   | 4                  | 0.99 \| 0.01 \| 0.0 \| 0.0                          | 0.421873   | 134.131823        | 0.353643       |
| 19 | CAH \(single linkage\) 5   | 5                  | 0.99 \| 0.01 \| 0.0 \| 0.0 \| 0.0                   | 0.416696   | 102.986591        | 0.345884       |
| 20 | CAH \(complete linkage\) 2 | 2                  | 0.99 \| 0.01                                        | 0.756695   | 391.532573        | 0.305244       |
| 21 | CAH \(complete linkage\) 3 | 3                  | 0.95 \| 0.05 \| 0.01                                | 0.369943   | 343.169567        | 0.827601       |
| 22 | CAH \(complete linkage\) 4 | 4                  | 0.93 \| 0.05 \| 0.02 \| 0.01                        | 0.393543   | 356.857502        | 0.783894       |
| 23 | CAH \(complete linkage\) 5 | 5                  | 0.92 \| 0.05 \| 0.02 \| 0.01 \| 0.01                | 0.398385   | 301.213019        | 0.790088       |
| 24 | OPTICS                     | 75                 | 0.65 \| 0.02 \| 0.01 \| 0.01 \| 0.01 \| 0.01 \| ... | -0.205192  | 15.975476         | 1.393156       |
| 25 | MeanShift                  | 14                 | 0.93 \| 0.03 \| 0.02 \| 0.01 \| 0.01 \| 0.0 \| ...  | 0.322275   | 95.081981         | 0.899153       |
| 26 | AffinityPropagation        | 128                | 0.03 \| 0.03 \| 0.03 \| 0.02 \| 0.02 \| ...         | 0.165159   | 91.454880         | 1.460425       |


In [ ]:
pd.DataFrame(
    [
        ["KMeans", "3 clusters"],
        ["GMM", "2 clusters"],
    ],
    columns=[
        "Algorithme de clustering",
        "Résultat",
    ],
)

In [ ]:
# todo: tableau pour dire quels clusters on choisit d'étudier

## Visualisation

In [ ]:
def affiche_taille_clusters(nom_cluster):
    plt.title("Taille des clusters")
    sns.histplot(df_avec_clusters[nom_cluster], shrink=0.5)

    plt.show()

In [ ]:
def affiche_clusters_acp(nom_cluster):
    _, ax = plt.subplots(1, 2, figsize=(12, 5))

    ax[0].set_title("Clusters sur les axes d'ACP 1-2")
    ax[1].set_title("Clusters sur les axes d'ACP 3-4")

    sns.scatterplot(
        composantes_acp,
        x="ACP1",
        y="ACP2",
        hue=df_avec_clusters[nom_cluster],
        alpha=0.8,
        ax=ax[0],
    )
    sns.scatterplot(
        composantes_acp,
        x="ACP3",
        y="ACP4",
        hue=df_avec_clusters[nom_cluster],
        alpha=0.8,
        ax=ax[1],
    )

    plt.show()

In [ ]:
def affiche_clusters_acm(nom_cluster):
    _, ax = plt.subplots(1, 2, figsize=(12, 5))

    ax[0].set_title("Clusters sur les axes d'ACM 1-2")
    ax[1].set_title("Clusters sur les axes d'ACM 3-4")

    sns.scatterplot(
        composantes_acm,
        x="ACM1",
        y="ACM2",
        hue=df_avec_clusters[nom_cluster],
        alpha=0.8,
        ax=ax[0],
    )

    sns.scatterplot(
        composantes_acm,
        x="ACM3",
        y="ACM4",
        hue=df_avec_clusters[nom_cluster],
        alpha=0.8,
        ax=ax[1],
    )

    plt.show()

In [ ]:
def affiche_clusters_var_quanti(nom_cluster):
    """Affiche les variables quantitatives en fonction des clusters."""
    for var in var_numeriques:
        _, ax = plt.subplots(1, 2, figsize=(10, 3))

        sns.boxplot(
            x=df[var],
            y=df_avec_clusters[nom_cluster],
            width=0.25,
            ax=ax[0],
        )

        sns.histplot(
            x=df[var],
            kde=True,
            ax=ax[1],
            hue=df_avec_clusters[nom_cluster],
            stat="probability",
            common_norm=False,
        )

        plt.show()

In [ ]:
def affiche_clusters_var_quali(nom_cluster):
    """Affiche les variables qualitatives en fonction des clusters et vice-versa."""
    for var in var_categoriques_fe:
        _, ax = plt.subplots(1, 2, figsize=(10, 4))

        sns.histplot(
            x=df[var].astype(str),
            ax=ax[0],
            hue=df_avec_clusters[nom_cluster],
            multiple="dodge",
            shrink=0.5,
            common_norm=True,
        )

        sns.histplot(
            hue=df[var].astype(str),
            ax=ax[1],
            x=df_avec_clusters[nom_cluster],
            multiple="dodge",
            shrink=0.5,
            common_norm=True,
        )

        plt.show()

In [ ]:
def affiche_clusters(nom_cluster):
    """Affiche les variables en fonction des clusters."""
    affiche_taille_clusters(nom_cluster)
    affiche_clusters_acp(nom_cluster)
    affiche_clusters_acm(nom_cluster)

    affiche_clusters_var_quanti(nom_cluster)
    affiche_clusters_var_quali(nom_cluster)

In [ ]:
affiche_clusters("KMeans3")

In [ ]:
affiche_clusters("GMM2")

In [ ]:
affiche_clusters("KMeans3")

In [ ]:
affiche_clusters("CAH (Ward) 3")

In [ ]:
# todo: faire un tableau

In [ ]:
# todo: mettre proportion des clients types

Les clients "type" : (85%)
- niveau d'éducation de bac jusqu'à doctorat
- revenu le plus élevé
- 0 enfant en bas âge / entre 0 et 1 enfant adolescent
- ont accepté entre 0 et 4 campagnes marketing précédentes
- dépensent le plus
- la proportion d'acceptation des campagnes est beaucoup plus élevée
- visitent très peu le site Internet

Les clients qui achètent peu :
- niveau d'éducation de bac jusqu'à doctorat
- revenu moyen
- entre 0 et 3 enfants (bas-âge et adolescent)
- ont accepté entre 0 et 1 campagne marketing précédente
-

Les clients qui n'achètent pas : (2%)
- niveau d'éducation très faible (brevet)
- revenu le plus faible
- entre 0 et 1 enfant en bas âge et 0 enfant adolescent
- n'ont accepté aucune des campagnes marketing précédentes
- dépensent très peu
- visitent souvent le site
- âge le plus jeune (moyenne : naissance en 1980)

# Sauvegarde des données

In [ ]:
# todo: sauvegarder les clusters